**Pose Esimation: Model Hub**

In [ ]:
%useLatestDescriptors
%use kotlin-dl (0.5.1)

In [ ]:
import org.jetbrains.kotlinx.dl.api.summary.printSummary
import org.jetbrains.kotlinx.dl.impl.preprocessing.image.ImageConverter
import org.jetbrains.kotlinx.dl.onnx.inference.ONNXModelHub
import org.jetbrains.kotlinx.dl.onnx.inference.ONNXModels
import org.jetbrains.kotlinx.dl.visualization.swing.createDetectedObjectsPanel
import org.jetbrains.kotlinx.dl.visualization.swing.showFrame
import java.io.File
import javax.imageio.ImageIO

In [ ]:
val PATH_TO_IMAGE = "/Users/Alexey.Zinoviev/IdeaProjects/KotlinDL-for-KotlinConf2023/src/main/resources/datasets/poses/1.jpg"

val PATH_TO_PRETRAINED_MODELS = "../../../cache/pretrainedModels"

In [ ]:
val modelHub = ONNXModelHub(cacheDirectory = File(PATH_TO_PRETRAINED_MODELS))
val poseDetectionModel = ONNXModels.PoseDetection.MoveNetSinglePoseLighting.pretrainedModel(modelHub)

In [ ]:
poseDetectionModel.printSummary()

In [ ]:
 DISPLAY(ImageIO.read(File(PATH_TO_IMAGE)))

In [ ]:
val image = ImageConverter.toBufferedImage(File(PATH_TO_IMAGE))

val detectedPose = poseDetectionModel.detectPose(image)
poseDetectionModel.close()

In [ ]:
detectedPose.landmarks.forEach {
    println("Found ${it.label} with probability ${it.probability}")
}

detectedPose.edges.forEach {
    println("The ${it.label} starts at ${it.start.label} and ends with ${it.end.label}")
}

In [ ]:
import org.jetbrains.kotlinx.dl.api.inference.objectdetection.DetectedObject
import org.jetbrains.kotlinx.dl.api.summary.printSummary
import org.jetbrains.kotlinx.dl.impl.preprocessing.image.ImageConverter
import org.jetbrains.kotlinx.dl.onnx.inference.ONNXModelHub
import org.jetbrains.kotlinx.dl.onnx.inference.ONNXModels
import org.jetbrains.kotlinx.dl.visualization.swing.createDetectedObjectsPanel
import org.jetbrains.kotlinx.dl.visualization.swing.showFrame
import java.awt.BasicStroke
import java.awt.Color
import java.awt.Graphics2D
import java.awt.Stroke
import java.awt.image.BufferedImage
import java.io.File
import javax.imageio.ImageIO
import kotlin.math.abs


In [ ]:
import java.awt.Graphics2D.*
import java.awt.geom.Ellipse2D
import java.awt.geom.Line2D
import java.awt.geom.Rectangle2D
import java.awt.image.BufferedImage
import java.awt.FontMetrics
import java.awt.RenderingHints
import java.awt.Font

In [ ]:
import org.jetbrains.kotlinx.dl.api.inference.posedetection.DetectedPose

fun drawDetectedPose(
    image: File,
    detectedPose: DetectedPose
): BufferedImage {
    val bufferedImage = ImageIO.read(image)

    val newGraphics = bufferedImage.createGraphics()
    newGraphics.drawImage(bufferedImage, 0, 0, null)
    newGraphics.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)
    newGraphics as Graphics2D

    val width = bufferedImage.width
    val height = bufferedImage.height

    val landmarkColor: Color = Color.RED
    val edgeColor: Color = Color.MAGENTA

    newGraphics.stroke = BasicStroke(2f)
    newGraphics.color = edgeColor
    detectedPose.edges.forEach { (start, end, _, _) ->
        newGraphics.draw(
            Line2D.Float(
                width * start.x, height * start.y,
                width * end.x, height * end.y
            )
        )
    }

    val r = 3.0f
    newGraphics.color = landmarkColor
    detectedPose.landmarks.forEach { (x, y, _, _) ->
        newGraphics.fill(Ellipse2D.Float(width * x - r, height * y - r, 2 * r, 2 * r))
    }

    ImageIO.write(bufferedImage, "jpg", File("../../../serverFiles/detectedPose.jpg"))
    newGraphics.dispose()
    return bufferedImage
}

In [ ]:
val bufferedImageWithPoses = drawDetectedPose(File(PATH_TO_IMAGE), detectedPose)
DISPLAY(bufferedImageWithPoses)